In [9]:
load_model = 'efficientnet_b7_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [10]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


## load the efficientnet_b7 based model

In [11]:
efficientnet_b7_loaded = models.efficientnet_b7(pretrained=True)

for param in efficientnet_b7_loaded.parameters():
    param.requires_grad = True

efficientnet_b7_loaded.fc = nn.Linear(in_features=2560, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
efficientnet_b7_loaded = efficientnet_b7_loaded.to(device)



efficientnet_b7_loaded.load_state_dict(torch.load('efficientnet_b7_based_model2.pt'))
efficientnet_b7_loaded.eval()

Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-dcc49843.pth" to /home/rka73/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-dcc49843.pth


EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
            (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivatio

In [12]:
model = efficientnet_b7_loaded

In [24]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            if(top1_catid[i] <= 120):  
                predicted_breed = categories[top1_catid[i]]

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

per_class_accuracy

{'Pomeranian': 100.0,
 'basset': 77.77777777777779,
 'Japanese spaniel': 94.73684210526315,
 'Bouvier des Flandres': 86.66666666666667,
 'German shepherd': 68.75,
 'Brabancon griffon': 87.5,
 'Blenheim spaniel': 84.21052631578947,
 'kelpie': 81.25,
 'Shih Tzu': 86.36363636363636,
 'curly coated retriever': 87.5,
 'Lakeland terrier': 70.0,
 'Irish setter': 100.0,
 'German short haired pointer': 93.75,
 'flat coated retriever': 68.75,
 'Irish water spaniel': 86.66666666666667,
 'Dandie Dinmont': 94.44444444444444,
 'clumber': 86.66666666666667,
 'Yorkshire terrier': 70.58823529411765,
 'schipperke': 87.5,
 'briard': 87.5,
 'miniature poodle': 50.0,
 'chow': 100.0,
 'Saluki': 90.0,
 'boxer': 87.5,
 'Pekinese': 66.66666666666666,
 'komondor': 100.0,
 'Brittany spaniel': 81.25,
 'black and tan coonhound': 68.75,
 'Appenzeller': 43.75,
 'Eskimo dog': 13.333333333333334,
 'giant schnauzer': 62.5,
 'Old English sheepdog': 94.11764705882352,
 'Chihuahua': 81.25,
 'miniature schnauzer': 81.25,
 

In [25]:
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

82.18854610301977

### Top 10 Breeds with least accuracy

In [26]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 13.333333333333334),
 ('American Staffordshire terrier', 35.294117647058826),
 ('Walker hound', 37.5),
 ('Appenzeller', 43.75),
 ('miniature poodle', 50.0),
 ('toy terrier', 50.0),
 ('Lhasa', 52.63157894736842),
 ('Irish terrier', 58.82352941176471),
 ('Australian terrier', 60.0),
 ('giant schnauzer', 62.5)]

### Top 15 Breeds with most accuracy

In [27]:
k = 15
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Pomeranian', 100.0),
 ('Irish setter', 100.0),
 ('chow', 100.0),
 ('komondor', 100.0),
 ('keeshond', 100.0),
 ('Samoyed', 100.0),
 ('Rottweiler', 100.0),
 ('Boston bull', 100.0),
 ('Doberman', 100.0),
 ('Kerry blue terrier', 100.0),
 ('groenendael', 100.0),
 ('standard poodle', 100.0),
 ('Weimaraner', 100.0),
 ('Afghan hound', 100.0),
 ('Sealyham terrier', 100.0)]